In [53]:
# def load_OCR_json_files(folder_path):
#     file_list = sorted(os.listdir(folder_path))
#     all_reviews = []
#     for file_name in file_list:
#         if file_name.endswith('.json'):
#             file_path = os.path.join(folder_path, file_name)
#             with open(file_path, 'r', encoding='utf-8-sig') as file:
#                 data = json.load(file)
#                 all_reviews.append(data)
#     return all_reviews

In [54]:
# folder_path="OCR_json"
# df = load_OCR_json_files(folder_path)

In [55]:
import re
def extract_element(sentence):
    # 태깅 문장과 토픽 인식하는 정규식
    pattern = r"(?P<sentence>.+?)\s*\(\s*(?P<topic>.+?)\s*\)"

    # 정규식 컴파일
    regex = re.compile(pattern)
    tag_if = False
    
    # 매칭 및 값 추출
    match = regex.search(sentence)
    if match:
        full_sentence = match.group('sentence').rstrip(':').strip()
        topic = match.group('topic').strip()
        tag_if = True
    else:
        full_sentence="태그 없음"
        topic="토픽 없음"
    return tag_if, full_sentence, topic

In [56]:
from transformers import pipeline
model_name="paust/pko-t5-base"

def generate_text(pipe, text, num_return_sequences=5, max_length=200):
  text = f"{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]

def run_pipeline(model_path, text):
    nlg_pipeline = pipeline('text2text-generation', model=model_path, tokenizer=model_name, device='cpu')
    return (generate_text(nlg_pipeline, text, num_return_sequences=1, max_length=200)[0])

In [57]:
def bbox_match(total_bbox_list, sentence):
    unmatched_bbox_list = []
    sentence_bbox_list = []
    
    for bbox in total_bbox_list:
        if bbox['text'] in sentence:
            sentence_bbox_list.append(bbox)
        else:
            unmatched_bbox_list.append(bbox)
    
    return unmatched_bbox_list, sentence_bbox_list

In [58]:
import json
import pandas as pd
file_path=r'C:\Users\KHU\Desktop\tagging_seq2seq\OCR_json\araree 네모와 미니 맥세이프 보조배터리 5000mAh.json'
with open(file_path, 'r', encoding='utf-8-sig') as file:
    df=[json.load(file)]

In [59]:
import copy
matched_bbox_list = []

for index, row in enumerate(df):
    for i in range(len(row)):
        splited_list = row[i][0].split("\n")
        if type(row[i][1]) == list:
            total_bbox_list = row[i][2:]
        else:
            total_bbox_list = row[i][1:]
        total_bbox_list_to_save = copy.deepcopy(total_bbox_list)
        # 한 이미지의 문장마다 태그 정보 필요함    
        tag_result_list=[]
        for sentence in splited_list:
            # 분할된 문장의 단어와 이미지 내의 단어와 매칭 --> bbox 묶음 (sentence_bbox_list)
            unmatched_bbox_list, sentence_bbox_list = bbox_match(total_bbox_list, sentence)
            # 해당 문장의 태그 정보
            model_tag=run_pipeline('OCR_model.pth', sentence)
            tag_info, tagged_sentence, topic = extract_element(model_tag)
            tag_result={}
            # 태그가 있으면 토픽과 함께 dict로 저장 
            if tag_info:
                tag_result['text']=tagged_sentence
                tag_result['topic']=topic
                tagged_bbox_list=[]
                for tag in sentence_bbox_list:
                    if tag['text'] in tagged_sentence:
                        tagged_bbox_list.append(tag['bbox'])
                tag_result['bbox']=tagged_bbox_list
            else:
                # 태그가 없는 경우는 json에 저장안함
                continue
            tag_result_list.append(tag_result)
            # 다음 문장에 대해, 앞에서 매칭되지 않고 남은 bbox 리스트에서 다시 매칭하도록함
            total_bbox_list = unmatched_bbox_list
        save_list=[row[i][0], tag_result_list]
        for bbox in total_bbox_list_to_save:
            save_list.append(bbox)
        matched_bbox_list.append(save_list)

In [65]:
with open('test_json.json', 'w', encoding='utf-8-sig') as f:
    json.dump(matched_bbox_list, f, ensure_ascii=False)

In [62]:
df = pd.json_normalize(matched_bbox_list)

In [63]:
df

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,{},[{'text': '항상 들고 다니는 주머니 속의 비상발전소 nemowa 500om...,"{'text': 'ororee', 'bbox': [[370.0, 1579.0], [...","{'text': '항상', 'bbox': [[240.0, 1757.0], [286....","{'text': '들고 다니는', 'bbox': [[279.0, 1756.0], [...","{'text': '주머니', 'bbox': [[400.0, 1757.0], [459...","{'text': '속의', 'bbox': [[467.0, 1756.0], [509....","{'text': '비상발전소', 'bbox': [[509.0, 1756.0], [6...","{'text': 'nemowa', 'bbox': [[170.0, 1820.0], [...","{'text': '500omah mini magsafe power bank', 'b...",...,None,None,None,None,None,None,None,None,None,None
1,{},[{'text': '5000mah의 넉넉한 용량과 고속 무선충전까지 가능한 갖췄을 ...,"{'text': 'nemowa', 'bbox': [[52.0, 1424.0], [3...","{'text': '네모와는', 'bbox': [[51.0, 1528.0], [130...","{'text': '주머니에 쏙 들어가는 동급 최소형 미니', 'bbox': [[13...","{'text': '맥세이프', 'bbox': [[491.0, 1526.0], [57...","{'text': '보조배터리입니다', 'bbox': [[583.0, 1529.0],...","{'text': '5000mah의', 'bbox': [[50.0, 1564.0], ...","{'text': '넉넉한 용량과 고속 무선충전까지', 'bbox': [[175.0,...","{'text': '가능한', 'bbox': [[484.0, 1563.0], [547...",...,None,None,None,None,None,None,None,None,None,None
2,{},[{'text': '항상 가지고 다닐 수 있도록 작고 가벼운 사이즈 빠르게 충전할 ...,"{'text': '그런', 'bbox': [[192.0, 860.0], [296.0...","{'text': '상황을', 'bbox': [[318.0, 862.0], [473....","{'text': '대비해', 'bbox': [[496.0, 859.0], [665....","{'text': '항상', 'bbox': [[77.0, 1060.0], [119.0...","{'text': '가지고', 'bbox': [[120.0, 1060.0], [184...","{'text': '다닐', 'bbox': [[187.0, 1061.0], [231....","{'text': '수', 'bbox': [[233.0, 1062.0], [257.0...","{'text': '있도록', 'bbox': [[260.0, 1061.0], [322...",...,None,None,None,None,None,None,None,None,None,None
3,{},[{'text': '지금 보실 보조배터리는 당신이 지금까지 보지 못했던 예쁘고 강력...,"{'text': '지금', 'bbox': [[258.0, 1568.0], [367....","{'text': '보실', 'bbox': [[376.0, 1568.0], [482....","{'text': '보조배터리는', 'bbox': [[499.0, 1571.0], [...","{'text': '당신이', 'bbox': [[142.0, 1646.0], [297...","{'text': '지금까지', 'bbox': [[313.0, 1646.0], [51...","{'text': '보지', 'bbox': [[533.0, 1646.0], [639....","{'text': '못했던', 'bbox': [[651.0, 1647.0], [806...","{'text': '예쁘고', 'bbox': [[194.0, 1725.0], [351...",...,None,None,None,None,None,None,None,None,None,None
4,{},"[{'text': '휴대하기 정말 좋대', 'topic': '휴대성', 'bbox'...","{'text': '휴대하기', 'bbox': [[50.0, 1337.0], [254...","{'text': '정말', 'bbox': [[273.0, 1337.0], [382....","{'text': '좋대', 'bbox': [[387.0, 1338.0], [489....","{'text': '네모와는', 'bbox': [[51.0, 1442.0], [133...","{'text': '항상', 'bbox': [[132.0, 1444.0], [180....","{'text': '들 고', 'bbox': [[179.0, 1445.0], [223...","{'text': '다니기', 'bbox': [[222.0, 1444.0], [285...","{'text': '좋은', 'bbox': [[286.0, 1445.0], [329....",...,None,None,None,None,None,None,None,None,None,None
5,{},"[{'text': '작고 가볍고 예쁘다!', 'topic': '무게', 'bbox'...","{'text': '작고', 'bbox': [[47.0, 2058.0], [155.0...","{'text': '가볍고', 'bbox': [[170.0, 2060.0], [322...","{'text': '예쁘다!', 'bbox': [[342.0, 2058.0], [50...","{'text': '네모와는', 'bbox': [[51.0, 2175.0], [131...","{'text': '동급', 'bbox': [[128.0, 2175.0], [174....","{'text': '보조배터리', 'bbox': [[178.0, 2175.0], [2...","{'text': '대비', 'bbox': [[280.0, 2174.0], [320....","{'text': '압도적으로', 'bbox': [[324.0, 2174.0], [4...",...,None,None,None,None,None,None,None,None,None,None
6,{},"[{'text': '네모와를 충전할 때도 빠르게 !', 'topic': '충전', ...","{'text': '네모와를', 'bbox': [[133.0, 1791.0], [33...","{'text': '충전할', 'bbox': [[349.0, 1791.0], [511...","{'text': '때도', 'bbox': [[521.0, 1791.0], [625....","{'text': '빠르게 !', 'bbox': [[636.0, 1789.0], [8...","{'text': '보조배터리를', 'bbox': [[417.0, 1903.0], [...","{'text': '충전할 때', 'bbox': [[541.0, 1903.0], [6...","{'text': '시간이', 'bbox': [[629.0, 1903.0], [690...","{'text': '오래 걸리죠?', 'bbox': [[693.0, 1903.0], ...",...,None,None,None,None,None,None,None,None,None,None
7,{},[],"{'text': '패스스루', 'bbox': [[168.0, 1567.0], [37...","{'text': '충전', 'bbox': [[388.0, 1565.0], [495....","{'text': '되나요?', 'bbox': [[508.0, 1568.0], [69...","{'text': '네', 'bbox': [[303.0, 1643.0], [368.0...","{'text': '됩니다!', 'bbox': [[386.0, 1647.0], [55.